# Hugging Face Transformers 微调训练入门

本示例将介绍基于 Transformers 实现模型微调训练的主要流程，包括：
- 数据集下载
- 数据预处理
- 训练超参数配置
- 训练评估指标设置
- 训练器基本介绍
- 实战训练
- 模型保存

## YelpReviewFull 数据集

**Hugging Face 数据集：[ YelpReviewFull ](https://huggingface.co/datasets/yelp_review_full)**

### 数据集摘要

Yelp评论数据集包括来自Yelp的评论。它是从Yelp Dataset Challenge 2015数据中提取的。

### 支持的任务和排行榜
文本分类、情感分类：该数据集主要用于文本分类：给定文本，预测情感。

### 语言
这些评论主要以英语编写。

### 数据集结构

#### 数据实例
一个典型的数据点包括文本和相应的标签。

来自YelpReviewFull测试集的示例如下：

```json
{
    'label': 0,
    'text': 'I got \'new\' tires from them and within two weeks got a flat. I took my car to a local mechanic to see if i could get the hole patched, but they said the reason I had a flat was because the previous patch had blown - WAIT, WHAT? I just got the tire and never needed to have it patched? This was supposed to be a new tire. \\nI took the tire over to Flynn\'s and they told me that someone punctured my tire, then tried to patch it. So there are resentful tire slashers? I find that very unlikely. After arguing with the guy and telling him that his logic was far fetched he said he\'d give me a new tire \\"this time\\". \\nI will never go back to Flynn\'s b/c of the way this guy treated me and the simple fact that they gave me a used tire!'
}
```

#### 数据字段

- 'text': 评论文本使用双引号（"）转义，任何内部双引号都通过2个双引号（""）转义。换行符使用反斜杠后跟一个 "n" 字符转义，即 "\n"。
- 'label': 对应于评论的分数（介于1和5之间）。

#### 数据拆分

Yelp评论完整星级数据集是通过随机选取每个1到5星评论的130,000个训练样本和10,000个测试样本构建的。总共有650,000个训练样本和50,000个测试样本。

## 下载数据集

In [1]:
from datasets import load_dataset

# Load full YelpReviewFull (train=650k, test=50k) and create a 5% validation split from train
raw_ds = load_dataset("yelp_review_full")
raw_splits = raw_ds["train"].train_test_split(test_size=0.05, seed=42)
ds = {
    "train": raw_splits["train"],
    "validation": raw_splits["test"],
    "test": raw_ds["test"]
}

num_labels = 5  # labels: 0..4 represent stars 1..5
print(ds["train"], ds["validation"], ds["test"])


/home/liaoq/miniconda3/envs/peftNew/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['label', 'text'],
    num_rows: 617500
}) Dataset({
    features: ['label', 'text'],
    num_rows: 32500
}) Dataset({
    features: ['label', 'text'],
    num_rows: 50000
})


In [2]:
ds

{'train': Dataset({
     features: ['label', 'text'],
     num_rows: 617500
 }),
 'validation': Dataset({
     features: ['label', 'text'],
     num_rows: 32500
 }),
 'test': Dataset({
     features: ['label', 'text'],
     num_rows: 50000
 })}

In [3]:
ds["train"][111]

{'label': 0,
 'text': "To whom it may concern, (which isn't anyone, because obviously NO ONE IS CONCERNED!) \\nWhen I called customer service the day my personalized order was delivered with some other child's name on it, I was told I would not have to send it back, that you don't take back personalized orders. Now you're telling me I have to send it back in order to get my refund? You know what, you can keep your refund. You obviously need the $20. more than I do. My family and friends all got a good laugh on Christmas morning due to your stupid mistake. I am not going to waste another second of my time dealing with Walmart. It's obvious to me now that Walmart doesn't care about it's customers. I will never order online from Walmart again, nor will I ever step foot in one of your stores!\\nSincerely yours,\\nLisa Hickey"}

In [4]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

In [5]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [6]:
show_random_elements(ds["train"])

,label,text
0,1 star,"The prices at this place are ridiculous. They charge $4.50 for 1 pancake and the quality is sub-standard, several tiers below grocery store pancake mix; in the category of a failed home cooking experiment. Even if I was a billionaire I couldn't eat here as it would entail a contradiction; you don't get what you pay for, plain and simple. \n\nIt's a sad state when the supply and demand market equilibrium of the Mesa restaurant economy allows this place to stay in business; perhaps people in Mesa, AZ have low culinary standards, but I think people should demand more for their money."
1,4 stars,"Yummy, yummy, yummy! I had crispy, greasy, packed with meat tacos just like I like `em. The beans and rice were great and the chips are the thinnest, most crispy delights I've had in a long time!\nThe staff was super nice too.\nThis place is super casual and good."
2,1 star,"Received a hat as a gift that was purchased from this store. Sadly hat sizes are very hit & miss. Went in to exchange the size and was told that I can't even exchange it for a different size without a receipt. The guy I talked to was very rude and said, \""how do we know you didn't buy it somewhere else?\"" Um how about the tag that says Cactus Sports?? The hat has all tags and is obviously brand new. Horrible customer service, don't spend money here."
3,1 star,"Horrible, horrible ...I can't write everything I've heard about this place, because it'd be slanderous and probably hard to believe anyway. But beware of everything if you do get dragged here."
4,5 stars,"Nice kitchen, spacious living area. Clean facility and great staff. Stayed 5 nights. Breakfast was good enough with waffle machines, and nice hot foods. Within walking distance to everything you would want to do, but if not, the metro is 1/2 a block away. Sweet roof top terrace near laundry room, with grand views of the city. Hope to stay here again."
5,2 star,"I really want to like this place, but in four different trips there the experience has mostly been bad. First time a bunch of us went for the rooftop bar, only to find it was closed for a private party. So we went to the lobby bar and the service was incredibly slow. We came back for the rooftop bar another weekend and actually got in...atmosphere is great! They could really do something with that, but unfortunately the 'portable' bars are terrible. Very slow service. Next time we came for rooftop bar and it was again a private party and we were turned away (never good to turn customers away!). Finally, we recommended some friends from L.A. stay there and the air conditioning was not working in the hotel. They moved them to FOUR different rooms and none of them had working air. As much as I want to like it, they can't seem to do much of anything to impress."
6,1 star,"The huff and irritated sigh that I was greeted with when calling Cure an hour before my reservation sums up this place: Snooty and pretentious with no reason to be. I simply wanted to clarify their BYOB or full bar status, which did seem to be a legitimate question. The young lady on the other end of the phone clearly did not agree. \""Sir, we've had a fully stocked bar for months now\"" she blurted at me in the snarkiest of snarky tones. Their idea of a fully stocked bar was three gins and three vodkas I'd never heard of before (and my name is not Clampett) and $12 for a gin-tonic. Plus a draft beer system that \""hasn't worked since we put it in.\"" Full bar!! Food? Not even worth mentioning. The owner and staff clearly expect us to be impressed and fall at their feet for the mere fact that they breathe while I expect to be treated with -- oh maybe the slightest hint of -- human kindness and sincerity. Not here. And I'm sure not ever. I will continue to venture to upper Lawrenceville to dine at Pusadee's Garden (best thai food in the city and best outdoor dining space) across the street, and I will dance a little jig in Butler Street the day I see this place shutte

## 预处理数据

下载数据集到本地后，使用 Tokenizer 来处理文本，对于长度不等的输入数据，可以使用填充（padding）和截断（truncation）策略来处理。

Datasets 的 `map` 方法，支持一次性在整个数据集上应用预处理函数。

下面使用填充到最大长度的策略，处理整个数据集：

In [7]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-uncased"  # keep BERT; you can switch to 'roberta-base' for potentially higher accuracy
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

max_length = 256  # increase to 384/512 if VRAM allows (slower but can help accuracy)
def tokenize_fn(example):
    return tokenizer(example["text"], truncation=True, max_length=max_length)

tokenized = {}
for split in ["train", "validation", "test"]:
    tokenized[split] = ds[split].map(tokenize_fn, batched=True, remove_columns=["text"])

for name in tokenized:
    tokenized[name].set_format(type="torch")
print(tokenized["train"])


/home/liaoq/miniconda3/envs/peftNew/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 617500
})


In [8]:
show_random_elements(ds["train"], num_examples=1)

,label,text
0,2 star,"Atmosphere is obnoxiously loud and bright with the overhead lights and the blaring TVs. Burgers were good, but nothing about makes them stand out amongst all the $$$ burger joints. Service was slow."


## 微调训练配置

### 加载 BERT 模型

警告通知我们正在丢弃一些权重（`vocab_transform` 和 `vocab_layer_norm` 层），并随机初始化其他一些权重（`pre_classifier` 和 `classifier` 层）。在微调模型情况下是绝对正常的，因为我们正在删除用于预训练模型的掩码语言建模任务的头部，并用一个新的头部替换它，对于这个新头部，我们没有预训练的权重，所以库会警告我们在用它进行推理之前应该对这个模型进行微调，而这正是我们要做的事情。

In [9]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 训练超参数（TrainingArguments）

完整配置参数与默认值：https://huggingface.co/docs/transformers/v4.36.1/en/main_classes/trainer#transformers.TrainingArguments

源代码定义：https://github.com/huggingface/transformers/blob/v4.36.1/src/transformers/training_args.py#L161

**最重要配置：模型权重保存路径(output_dir)**

In [10]:
from transformers import TrainingArguments

model_dir = "models/bert-base-cased-finetune-yelp"

# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为100
training_args = TrainingArguments(output_dir=model_dir,
                                  per_device_train_batch_size=16,
                                  num_train_epochs=5,
                                  logging_steps=100)

In [11]:
# 完整的超参数配置
print(training_args)

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
gradient_checkpointing_kwargs=None,
greater_is_better=None,
group_by_le

### 训练过程中的指标评估（Evaluate)

**[Hugging Face Evaluate 库](https://huggingface.co/docs/evaluate/index)** 支持使用一行代码，获得数十种不同领域（自然语言处理、计算机视觉、强化学习等）的评估方法。 当前支持 **完整评估指标：https://huggingface.co/evaluate-metric**

训练器（Trainer）在训练过程中不会自动评估模型性能。因此，我们需要向训练器传递一个函数来计算和报告指标。 

Evaluate库提供了一个简单的准确率函数，您可以使用`evaluate.load`函数加载

In [12]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")


接着，调用 `compute` 函数来计算预测的准确率。

在将预测传递给 compute 函数之前，我们需要将 logits 转换为预测值（**所有Transformers 模型都返回 logits**）。

In [13]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

#### 训练过程指标监控

通常，为了监控训练过程中的评估指标变化，我们可以在`TrainingArguments`指定`evaluation_strategy`参数，以便在 epoch 结束时报告评估指标。

In [14]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=model_dir,
                                  evaluation_strategy="epoch", 
                                  per_device_train_batch_size=16,
                                  num_train_epochs=3,
                                  logging_steps=30)

## 开始训练

### 实例化训练器（Trainer）

`kernel version` 版本问题：暂不影响本示例代码运行

In [15]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {"accuracy": accuracy.compute(predictions=preds, references=p.label_ids)["accuracy"]}

model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels
)

train_args = TrainingArguments(
    output_dir="yelp_full_bert",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",

    per_device_train_batch_size=16,   # lower to 8/4 if OOM
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=2,    # increase if VRAM is small
    num_train_epochs=3,
    learning_rate=2e-5,
    warmup_ratio=0.06,
    weight_decay=0.01,

    fp16=True,                        # set to False on CPU/unsupported GPUs
    report_to="none",
    seed=42
)

trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer


/home/liaoq/miniconda3/envs/peftNew/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 使用 nvidia-smi 查看 GPU 使用

为了实时查看GPU使用情况，可以使用 `watch` 指令实现轮询：`watch -n 1 nvidia-smi`:

```shell
Every 1.0s: nvidia-smi                                                   Wed Dec 20 14:37:41 2023

Wed Dec 20 14:37:41 2023
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:0D.0 Off |                    0 |
| N/A   64C    P0              69W /  70W |   6665MiB / 15360MiB |     98%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+

+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|
|    0   N/A  N/A     18395      C   /root/miniconda3/bin/python                6660MiB |
+---------------------------------------------------------------------------------------+
```

In [16]:
# Train and evaluate on the validation split
train_result = trainer.train()
metrics_val = trainer.evaluate()
print("Validation metrics:", metrics_val)

# Save best model
trainer.save_model("yelp_full_bert/best")


Epoch,Training Loss,Validation Loss,Accuracy
1,0.724700,0.723093,0.684308
2,0.676100,0.715099,0.693200
3,0.604200,0.733796,0.694523


Validation metrics: {'eval_loss': 0.7337957620620728, 'eval_accuracy': 0.6945230769230769, 'eval_runtime': 147.4793, 'eval_samples_per_second': 220.37, 'eval_steps_per_second': 6.889, 'epoch': 3.0}


In [17]:
# Final evaluation on the test set
test_metrics = trainer.evaluate(eval_dataset=tokenized["test"])
print("Test metrics:", test_metrics)


Test metrics: {'eval_loss': 0.7413249015808105, 'eval_accuracy': 0.68976, 'eval_runtime': 225.7886, 'eval_samples_per_second': 221.446, 'eval_steps_per_second': 6.922, 'epoch': 3.0}


In [19]:
small_test_dataset = tokenized["test"].shuffle(seed=64).select(range(100))

In [20]:
trainer.evaluate(small_test_dataset)

{'eval_loss': 0.9016115665435791,
 'eval_accuracy': 0.66,
 'eval_runtime': 0.4994,
 'eval_samples_per_second': 200.248,
 'eval_steps_per_second': 8.01,
 'epoch': 3.0}

### 保存模型和训练状态

- 使用 `trainer.save_model` 方法保存模型，后续可以通过 from_pretrained() 方法重新加载
- 使用 `trainer.save_state` 方法保存训练状态

In [21]:
trainer.save_model(model_dir)

In [22]:
trainer.save_state()

In [ ]:
# trainer.model.save_pretrained("./")

## Homework: 使用完整的 YelpReviewFull 数据集训练，看 Acc 最高能到多少

通过3个epoch的全量数据训练，最高准确率为0.694523，平均准确率为0.66！！